In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import json
from src import preparedata
from src import modelarea
from src import trainarea

print(tf.__version__)
print(tf.config.list_physical_devices(
    device_type=None
))

params=json.load(open('params/params.json','r'))

2023-01-18 15:25:34.451302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 15:25:34.604275: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-18 15:25:34.657373: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-18 15:25:39.547042: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

2.10.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:

dataset=preparedata.readGPDData(params['dataprepinargs'])
dataset.preparedata()

  0%|          | 0/31 [00:00<?, ?it/s]

In [3]:
landslidehazard=modelarea.lhmodel(params['modelparam'])
landslidehazard.preparemodel()

2023-01-18 15:27:28.774946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 15:27:29.303824: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30974 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:3d:00.0, compute capability: 7.0


In [4]:
dataset.X_train.shape

(2035968, 29)

In [5]:
 import numpy as np
 np.isnan(dataset.Y_train).any()

False

In [6]:
trainarea.trainmodel(landslidehazard.model,dataset.X_train,dataset.Y_train,params['trainparam'])

Epoch 1/100
2784/2784 [==============================] - ETA: 0s - loss: 0.0966 - gpdmetric: 0.9377
Epoch 1: val_loss improved from inf to 0.00997, saving model to checkpoints/
INFO:tensorflow:Assets written to: checkpoints/assets
2784/2784 [==============================] - 42s 14ms/step - loss: 0.0966 - gpdmetric: 0.9377 - val_loss: 0.0100 - val_gpdmetric: 0.9955
Epoch 2/100
2784/2784 [==============================] - ETA: 0s - loss: 0.0135 - gpdmetric: 0.9931
Epoch 2: val_loss improved from 0.00997 to 0.00965, saving model to checkpoints/
INFO:tensorflow:Assets written to: checkpoints/assets
2784/2784 [==============================] - 37s 13ms/step - loss: 0.0135 - gpdmetric: 0.9931 - val_loss: 0.0097 - val_gpdmetric: 0.9955
Epoch 3/100
2784/2784 [==============================] - ETA: 0s - loss: 0.0108 - gpdmetric: 0.9951
Epoch 3: val_loss improved from 0.00965 to 0.00962, saving model to checkpoints/
INFO:tensorflow:Assets written to: checkpoints/assets
2784/2784 [==============

KeyboardInterrupt: 

In [ ]:
y=landslidehazard.model.predict(dataset.X_train[:10,:])

1/1 [==============================] - 1s 1s/step


In [13]:
np.array(dataset.Y_train>0.0,dtype=int).sum()/np.array(dataset.Y_train>0.0,dtype=int).shape[0]


0.010034047686407645

In [ ]:
import tensorflow_probability as tfp 
tfd = tfp.distributions


In [109]:
dist=tfp.distributions.GeneralizedExtremeValue(loc=0.0,scale=0.1,concentration=0.0,validate_args=False,allow_nan_stats=False,name='GeneralizedExtremeValue')
        

In [15]:
aaa=np.array([1,2,3])

In [16]:
aaa[aaa==2]=5

In [17]:
aaa

array([1, 5, 3])